In [12]:
import pandas as pd
import tensorflow as tf
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from keras.models import Sequential,load_model
from keras.layers import LSTM, Dense,Dropout,Bidirectional
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
tf.get_logger().setLevel('ERROR')

In [27]:
df = pd.read_csv('../dataset/global_dataset.csv') 
print(df.info())


X =df[['TaskFileSize', 'TaskOutputFileSize', 'TaskFileLength', 'CpuTime','TotalLength']]  # Features
y=df['VmID'] # Labels
unique_labels = np.unique(df['VmID'])
num_classes=len(unique_labels)
print(unique_labels)
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   TaskID                 2000 non-null   int64  
 1   TaskFileSize           2000 non-null   int64  
 2   TaskOutputFileSize     2000 non-null   int64  
 3   TaskFileLength         2000 non-null   int64  
 4   CpuTime                2000 non-null   float64
 5   TotalLength            2000 non-null   int64  
 6   UserLatitude           2000 non-null   float64
 7   UserLongitude          2000 non-null   float64
 8   DataCenterID           2000 non-null   int64  
 9   VmID                   2000 non-null   int64  
 10  ENSEMBLE_predicted_DC  2000 non-null   int64  
 11  GA_predicted_DC        2000 non-null   int64  
 12  SNAKE_predicted_DC     2000 non-null   int64  
dtypes: float64(3), int64(10)
memory usage: 203.2 KB
None
[ 2  3  4  5  6  7  8  9 10 11 12 13 14 15]
<class 'pan

In [28]:

print(np.unique(y))

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(np.unique(y))
print(X.info())



[ 2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   TaskFileSize        2000 non-null   int64  
 1   TaskOutputFileSize  2000 non-null   int64  
 2   TaskFileLength      2000 non-null   int64  
 3   CpuTime             2000 non-null   float64
 4   TotalLength         2000 non-null   int64  
dtypes: float64(1), int64(4)
memory usage: 78.2 KB
None


In [29]:
if (len(X_train.shape)<3):
    X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))
print(" xtrain ", X_train.shape, "\n", "ytrain ", y_train.shape, "\n", "xtest  ", X_test.shape, "\n", "ytest ", y_test.shape, "\n")




 xtrain  (1600, 1, 5) 
 ytrain  (1600,) 
 xtest   (400, 1, 5) 
 ytest  (400,) 



In [30]:
def trainModel(hyperparameters):
    # Build and train the model
    model = Sequential()
    model.add(LSTM(units=int(hyperparameters['units']), input_shape=(1, X_train.shape[2]), return_sequences=True))
    model.add(Dropout(hyperparameters['dropout_rate']))
    model.add(LSTM(units=int(hyperparameters['units']) // 2))
    model.add(Dense(units=num_classes, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=hyperparameters['learning_rate']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    hist = model.fit(X_train, y_train,
                        epochs=100,
                        batch_size=int(hyperparameters['batch_size']),
                        validation_data=(X_test, y_test),
                       )
    return model,hist

SNAKE_hyperparameters = {'units': 128, 'dropout_rate': 0.01, 'learning_rate': 0.001, 'batch_size': 32}

snake_model,snake_hist= trainModel(SNAKE_hyperparameters)


Epoch 1/100
50/50 [==============================] - 5s 27ms/step - loss: 2.2349 - accuracy: 0.2844 - val_loss: 1.9474 - val_accuracy: 0.3300
Epoch 2/100
50/50 [==============================] - 0s 7ms/step - loss: 1.9064 - accuracy: 0.3156 - val_loss: 1.8767 - val_accuracy: 0.3450
Epoch 3/100
50/50 [==============================] - 0s 7ms/step - loss: 1.8383 - accuracy: 0.3413 - val_loss: 1.8748 - val_accuracy: 0.3175
Epoch 4/100
50/50 [==============================] - 0s 7ms/step - loss: 1.7806 - accuracy: 0.3781 - val_loss: 1.7792 - val_accuracy: 0.4550
Epoch 5/100
50/50 [==============================] - 0s 7ms/step - loss: 1.6319 - accuracy: 0.4956 - val_loss: 1.5493 - val_accuracy: 0.5125
Epoch 6/100
50/50 [==============================] - 0s 7ms/step - loss: 1.3960 - accuracy: 0.5525 - val_loss: 1.3468 - val_accuracy: 0.5900
Epoch 7/100
50/50 [==============================] - 0s 7ms/step - loss: 1.2283 - accuracy: 0.5913 - val_loss: 1.2364 - val_accuracy: 0.5625
Epoch 8/100


In [31]:
snake_accuracy = max(snake_hist.history["accuracy"])

print("snake  Accuracy: ",np.round(snake_accuracy* 100) ,'%')



snake  Accuracy:  99.0 %


In [33]:
snake_model.save('../models/snake_vms_scheduling_99.keras')